In [ ]:
# !git clone https://github.com/ankanbhunia/Handwriting-Transformers
# %cd Handwriting-Transformers
# !pip install --upgrade --no-cache-dir gdown
# !gdown --id 16g9zgysQnWk7-353_tMig92KsZsrcM6k && unzip files.zip && rm files.zip

In [1]:
import os
import time
from data.dataset import TextDataset, TextDatasetval
import torch
import cv2
import os
import numpy as np
from models.model import TRGAN
from params import *
from torch import nn
from data.dataset import get_transform
import pickle
from PIL import Image
import tqdm
import shutil

In [2]:
text = "동해물과 백두산이 마르고 닳도록"
output_path = 'results0524'

In [3]:
model_path = 'saved_models\hwt_kor_lr0.00005\model10.pth'; data_path = 'files_kr\korean_handwriting.pkl' #(iam)
#model_path = 'files/cvl_model.pth'; data_path = 'files/CVL-32.pickle' #(cvl)
#model_path = 'files/iam_model.pth'; data_path = 'files/CVL-32.pickle' #(iam-cvl-cross)
#model_path = 'files/cvl_model.pth'; data_path = 'files/IAM-32.pickle' #(cvl-iam-cross)#

In [6]:
print ('(1) Loading dataset files...')

TextDatasetObjval = TextDatasetval(base_path = data_path, num_examples = 1)
datasetval = torch.utils.data.DataLoader(
            TextDatasetObjval,
            batch_size=batch_size,
            shuffle=True,
            num_workers=0,
            pin_memory=True, drop_last=True,
            collate_fn=TextDatasetObjval.collate_fn)

print ('(2) Loading model...')

model = TRGAN()
# model.netG.load_state_dict(torch.load(model_path))
state = torch.load(model_path)    # TRGAN 전체의 state_dict
model.load_state_dict(state)

print (model_path+' : Model loaded Successfully')

print ('(3) Loading text content...')
text_encode =  [j.encode() for j in text.split(' ')]
eval_text_encode, eval_len_text = model.netconverter.encode(text_encode)
eval_text_encode = eval_text_encode.to('cuda:0').repeat(batch_size, 1, 1)


(1) Loading dataset files...
(2) Loading model...
initialize network with N02
initialize network with N02
initialize network with N02


C:\Users\User\AppData\Local\Temp\ipykernel_21856\3726354706.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(model_path)    # TRGAN 전체의 state_dict


saved_models\hwt_kor_lr0.00005\model10.pth : Model loaded Successfully
(3) Loading text content...


In [7]:

if os.path.isdir(output_path): shutil.rmtree(output_path)
os.makedirs(output_path, exist_ok = True)

for i,data_val in enumerate(tqdm.tqdm(datasetval)): 

    page_val = model._generate_page(data_val['simg'].to(DEVICE), data_val['swids'], eval_text_encode,eval_len_text)

    cv2.imwrite(output_path+'/image' + str(i) + '.png', page_val*255)

print ('\nOutput images saved in : ' + output_path)

100%|██████████| 3/3 [00:00<00:00,  8.94it/s]


Output images saved in : results0524
